In [114]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/int20h-main-2025/sample_submission.csv
/kaggle/input/int20h-main-2025/merchant_8508273933093481573/chargebacks.csv
/kaggle/input/int20h-main-2025/merchant_8508273933093481573/fraud_alerts.csv
/kaggle/input/int20h-main-2025/merchant_8508273933093481573/orders/20240101-20240128.csv
/kaggle/input/int20h-main-2025/merchant_8508273933093481573/orders/20230814-20230910.csv
/kaggle/input/int20h-main-2025/merchant_8508273933093481573/orders/20240520-20240616.csv
/kaggle/input/int20h-main-2025/merchant_8508273933093481573/orders/20231009-20231105.csv
/kaggle/input/int20h-main-2025/merchant_8508273933093481573/orders/20231106-20231203.csv
/kaggle/input/int20h-main-2025/merchant_8508273933093481573/orders/20230619-20230716.csv
/kaggle/input/int20h-main-2025/merchant_8508273933093481573/orders/20230601-20230618.csv
/kaggle/input/int20h-main-2025/merchant_8508273933093481573/orders/20230717-20230813.csv
/kaggle/input/int20h-main-2025/merchant_8508273933093481573/orders/20240129-202402

In [115]:
import glob
import warnings
warnings.simplefilter("ignore")
pd.set_option("display.max_columns", None)

### Add merchant_id to download all the information with his id

In [116]:
merchant_id = "8558295590389402168"
path = f"/kaggle/input/int20h-main-2025/merchant_{merchant_id}/orders/*.csv"
chargebacks_path = f"/kaggle/input/int20h-main-2025/merchant_{merchant_id}/chargebacks.csv"
fraud_alerts_path = f"/kaggle/input/int20h-main-2025/merchant_{merchant_id}/fraud_alerts.csv"

In [117]:
order_files = glob.glob(path)
df_ops = pd.concat([pd.read_csv(f) for f in order_files], ignore_index=True)
df_chargebacks = pd.read_csv(chargebacks_path)
df_fraud_alerts = pd.read_csv(fraud_alerts_path)

### Change column names to avoid name conflicts

In [118]:
df_chargebacks = df_chargebacks.rename(columns={
    "description": "chargeback_description", 
    "reason_code": "chargeback_reason_code", 
    "created_at": "chargeback_creation_time"})
df_fraud_alerts = df_fraud_alerts.rename(columns={
    "description": "alert_description", 
    "reason_code": "alert_reason_code",
    "created_at": "alert_creation_time"})

### Merge tables to keep information together

In [119]:
df = df_ops.merge(df_chargebacks, on=["transaction_id"], how="left").merge(df_fraud_alerts, on=["transaction_id"], how="left")

### created_at -> to_datetime()

In [120]:
df["created_at"] = pd.to_datetime(df["created_at"])
df["chargeback_creation_time"] = pd.to_datetime(df["chargeback_creation_time"])
df["alert_creation_time"] = pd.to_datetime(df["alert_creation_time"])

### Retrieve all information that we need for our risk_metric

In [121]:
new_df = df[["merchant_id","created_at", "transaction_id", "chargeback_creation_time","chargeback_reason_code","chargeback_description","chargeback_group","alert_creation_time","alert_reason_code","alert_description"]].iloc[:,:]

### Group rows by (date in day format)

In [122]:
new_df["created_at"] = pd.to_datetime(new_df["created_at"]).dt.date
new_df_grouped = new_df[["created_at", "transaction_id", "chargeback_creation_time", "alert_creation_time"]].groupby("created_at").count()
new_df_grouped = new_df_grouped.rename(columns={"transaction_id": "transaction_count", 
                                                "chargeback_creation_time": "chargeback_count",
                                               "alert_creation_time": "alert_count"})

### We calculate the formula (risk_metric) for each row

In [123]:
new_df_grouped = new_df_grouped.reset_index()
start_date = new_df_grouped["created_at"] - pd.Timedelta(days=90)
end_date = new_df_grouped["created_at"]
count_chargebacks_list = []
count_alerts_list = []
for sd, ed in zip(start_date, end_date):
    filtered_df = new_df_grouped[(new_df_grouped["created_at"] > sd) & (new_df_grouped["created_at"] <= ed)]
    if not filtered_df.empty:
        count_chargebacks = filtered_df["chargeback_count"].sum()
        count_alerts = filtered_df["alert_count"].sum()
    else:
        count_chargebacks = 0
        count_alerts = 0
    count_chargebacks_list.append(count_chargebacks)
    count_alerts_list.append(count_alerts)

new_counts_df = pd.DataFrame({"alert_count_90d": count_alerts_list,
                             "chargebacks_count_90d": count_chargebacks_list})

### Add this calculated DataFrame to main table we work with

In [124]:
new_df_grouped = pd.concat([new_df_grouped, new_counts_df], axis=1)

In [125]:
new_df_grouped["risk_metric"] = (new_df_grouped["alert_count_90d"] + new_df_grouped["chargebacks_count_90d"]) / new_df_grouped["transaction_count"]

### We retrieve rows from test set which have merchant_id we need

In [75]:
test_df = pd.read_csv("/kaggle/input/int20h-main-2025/sample_submission.csv")
test_df = test_df[test_df["merchant_id_day"].str.contains("8558295590389402168")]
test_df["date_parsed"] = test_df.iloc[:, 0].astype(str).apply(
    lambda x: pd.to_datetime(x.split("_", 1)[1]) if "_" in x else pd.NaT
)

In [77]:
test_df['date_parsed'] = pd.to_datetime(test_df['date_parsed'])
new_df_grouped["date_parsed"] = pd.to_datetime(new_df_grouped["created_at"]).dt.date

### For each row in test set we calculate values

In [78]:
risk_events = []

for i, row in test_df.iterrows():
    test_date = row['date_parsed']
    start_date = test_date - pd.Timedelta(days=90)
    filtered_train_df = new_df_grouped[(pd.to_datetime(new_df_grouped['created_at']) > start_date) & 
                                  (pd.to_datetime(new_df_grouped['created_at']) <= test_date)]
    if len(filtered_train_df) > 0:

        risk_events_last = filtered_train_df['risk_metric'].mean()
        risk_events.append(risk_events_last)
    else:

        risk_events.append(0)
test_df['predicted_risk_metric'] = risk_events
submission_df = test_df[['merchant_id_day', 'predicted_risk_metric']]

In [79]:
# risk_events = []
# count_chargebacks_list = []
# count_alerts_list = []
# count_transaction_list = []
# for i, row in test_df.iterrows():
#     test_date = row['date_parsed']

#     start_date = test_date - pd.Timedelta(days=30)
#     filtered_train_df = new_df_grouped[(pd.to_datetime(new_df_grouped['created_at']) > start_date) & 
#                                   (pd.to_datetime(new_df_grouped['created_at']) <= test_date)]
    
#     if len(filtered_train_df) > 0:
#         count_chargebacks = filtered_df["chargeback_count"].sum()
#         count_alerts = filtered_df["alert_count"].sum()
#         trans_count = filtered_df["transaction_count"].sum()
#         count_chargebacks_list.append(count_chargebacks)
#         count_alerts_list.append(count_alerts)
#         count_transaction_list.append(trans_count)
#     else:
#         count_chargebacks = 0
#         count_alerts = 0
#         trans_count = 0
#         count_chargebacks_list.append(count_chargebacks)
#         count_alerts_list.append(count_alerts)
#         count_transaction_list.append(trans_count)
# new_counts_df = pd.DataFrame({"alert_count_30d": count_alerts_list,
#                              "chargebacks_count_30d": count_chargebacks_list,
#                              "transaction_count_30d": count_transaction_list})

# test_df = pd.concat([test_df,new_counts_df],axis=1)
# # Додаємо прогноз в тестовий датасет
# test_df['risk_metric'] = (test_df["alert_count_30d"] + test_df["chargebacks_count_30d"]) / test_df["transaction_count_30d"]

# # Формуємо результуючий CSV для подачі
# submission_df = test_df[['merchant_id_day', 'risk_metric']]

### We need to run this notebook for every merchant and concatenate tables together

In [81]:
submission_df.to_csv('submission1.csv', index=False)